In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
from scipy.stats import linregress
from matplotlib.colors import LightSource
from matplotlib import cm
import pickle
#from landlab import RasterModelGrid, imshow_grid
import matplotlib as mpl
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy.signal import savgol_filter
from scipy.stats import linregress
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.gridspec import GridSpec

In [ ]:
exp1 = pd.read_csv('ell-experiments_Sc-09/ell-experiments_Sc-09_index.txt', delimiter = '\t')
exp1['slope'] = 2*exp1['cone_height']/exp1['cone_width']
exp1['l'] = exp1['D'] * exp1['S_c']/ exp1['b_0']

exp2 = pd.read_csv('ell-experiments_Sc-115/ell-experiments_Sc-115_index.txt', delimiter = '\t')
exp2['slope'] = 2*exp2['cone_height']/exp2['cone_width']
exp2['l'] = exp2['D'] * exp2['S_c']/ exp2['b_0']

exp3 = pd.read_csv('ell-experiments_Sc-14/ell-experiments_Sc-14_index.txt', delimiter = '\t')
exp3['slope'] = 2*exp3['cone_height']/exp3['cone_width']
exp3['l'] = exp3['D'] * exp3['S_c']/ exp3['b_0']


#Calculate power law best fit
m1, b1, p1, r1, se1 = linregress(np.log10(exp1['l']), np.log10(exp1['cone_height']))
m2, b2, p2, r2, se2 = linregress(np.log10(exp2['l']), np.log10(exp2['cone_height']))
m3, b3, p3, r3, se3 = linregress(np.log10(exp3['l']), np.log10(exp3['cone_height']))

m1s, b1s, p1s, r1s, se1s = linregress(np.log10(exp1['l']), np.log10(exp1['slope']))
m2s, b2s, p2s, r2s, se2s = linregress(np.log10(exp2['l']), np.log10(exp2['slope']))
m3s, b3s, p3s, r3s, se3s = linregress(np.log10(exp3['l']), np.log10(exp3['slope']))


#Calculate predictions and errors
exp1['predicted_height'] = 10**(mk*exp1['S_c'] + bk)*exp1['l']**(m*exp1['S_c'] + b)
exp2['predicted_height'] = 10**(mk*exp2['S_c'] + bk)*exp2['l']**(m*exp2['S_c'] + b)
exp3['predicted_height'] = 10**(mk*exp3['S_c'] + bk)*exp3['l']**(m*exp3['S_c'] + b)

exp1['predicted_slope'] = 10**(mks*exp1['S_c'] + bks)*exp1['l']**(ms*exp1['S_c'] + bs)
exp2['predicted_slope'] = 10**(mks*exp2['S_c'] + bks)*exp2['l']**(ms*exp2['S_c'] + bs)
exp3['predicted_slope'] = 10**(mks*exp3['S_c'] + bks)*exp3['l']**(ms*exp3['S_c'] + bs)

exp1['resid_heights'] = exp1['predicted_height']-exp1['cone_height']
exp1['resid_slope'] = exp1['predicted_slope'] - exp1['slope']

exp2['resid_heights'] = exp2['predicted_height']-exp2['cone_height']
exp2['resid_slope'] = exp2['predicted_slope'] - exp2['slope']

exp3['resid_heights'] = exp3['predicted_height']-exp3['cone_height']
exp3['resid_slope'] = exp3['predicted_slope'] - exp3['slope']

fig, ax = plt.subplots(2,2, figsize = (10, 9))

x = np.linspace(0.01, 1, 10)
ax[0,0].plot(x, 10**b1*x**m1, color = 'k', linestyle = 'dashed')
ax[0,0].plot(x, 10**b2*x**m2, color = 'k', linestyle = 'dashdot')
ax[0,0].plot(x, 10**b3*x**m3, color = 'k', linestyle = 'dotted')


sns.scatterplot(exp1, x = 'l', y = 'cone_height', ax = ax[0,0], label = '$S_c = 0.9$', c = '#E69F00', s = 70)
sns.scatterplot(exp2, x = 'l', y = 'cone_height', ax = ax[0,0], label = '$S_c = 1.15$', c = '#56B4E9', s = 70)
sns.scatterplot(exp3, x = 'l', y = 'cone_height', ax = ax[0,0], label = '$S_c = 1.4$', c = '#009E73', s = 70)

ax[0,0].set_xscale('log')
ax[0,0].set_yscale('log')

ax[0,0].tick_params(axis='y', which='minor', labelsize=0, labelcolor = 'white')
ax[0,0].set_axisbelow(True)
ax[0,0].set_xlabel('$\ell$ (m)', fontsize = 11)
ax[0,0].set_ylabel("$z'$ (m)", fontsize = 12)
ax[0,0].legend(framealpha = 1, loc = 'lower left')

x = np.linspace(0.01, 1, 10)
ax[1,0].plot(x, 10**b1s*x**m1s, color = 'k', linestyle = 'dashed')
ax[1,0].plot(x, 10**b2s*x**m2s, color = 'k', linestyle = 'dashdot')
ax[1,0].plot(x, 10**b3s*x**m3s, color = 'k', linestyle = 'dotted')

sns.scatterplot(exp1, x = 'l', y = 'slope', ax = ax[1,0], label = '$S_c = 0.9$', c = '#E69F00', s = 70, legend = True)
sns.scatterplot(exp2, x = 'l', y = 'slope', ax = ax[1,0], label = '$S_c = 1.15$', c = '#56B4E9', s = 70, legend = True)
sns.scatterplot(exp3, x = 'l', y = 'slope', ax = ax[1,0], label = '$S_c = 1.4$', c = '#009E73', s = 70, legend = True)

ax[1,0].set_xscale('log')
ax[1,0].set_yscale('log')
ax[1,0].set_axisbelow(True)
ax[1,0].set_xlabel('$\ell$ (m)', fontsize = 11)
ax[1,0].set_ylabel("$\\overline{\\theta}$ (m/m)", fontsize = 11)
ax[1,0].legend(framealpha = 1, loc = 'lower left')

"""
ztemp = np.linspace(0.3, 1.2)
sns.scatterplot(x = exp1['cone_height'], y = exp1['predicted_height'], ax = ax[0,1], label = '$S_c = 0.9$', c = '#E69F00', s = 70, legend = True)
sns.scatterplot(x = exp2['cone_height'], y = exp2['predicted_height'], ax = ax[0,1], label = '$S_c = 1.15$', c = '#56B4E9', s = 70, legend = True)
sns.scatterplot(x = exp3['cone_height'], y = exp3['predicted_height'], ax = ax[0,1], label = '$S_c = 1.4$', c = '#009E73', s = 70, legend = True)
ax[0,1].plot(ztemp, ztemp, color = 'k', linewidth = 1)
ax[0,1].set_xlabel('Cone Height (m)', fontsize = 12)
ax[0,1].set_ylabel('Predicted Height (m)', fontsize = 12)
"""

sns.scatterplot(x = exp1['l'], y = exp1['resid_heights'], ax = ax[0,1], label = '$S_c = 0.9$', c = '#E69F00', s = 70, legend = True)
sns.scatterplot(x = exp2['l'], y = exp2['resid_heights'], ax = ax[0,1], label = '$S_c = 1.15$', c = '#56B4E9', s = 70, legend = True)
sns.scatterplot(x = exp3['l'], y = exp3['resid_heights'], ax = ax[0,1], label = '$S_c = 1.4$', c = '#009E73', s = 70, legend = True)
ax[0,1].set_xlabel('$\\ell$ (m)', fontsize = 11)
ax[0,1].set_ylabel("Predicted Height Errors, $\\sigma_{z'}$ (m)", fontsize = 11)
ax[0,1].set_xlim([10**(-2)-0.001, 1.1])
ax[0,1].set_xscale('log')

"""
stemp = np.linspace(0.2, 1.2)
sns.scatterplot(x = exp1['slope'], y = exp1['predicted_slope'], ax = ax[1,1], label = '$S_c = 0.9$', c = '#E69F00', s = 70, legend = True)
sns.scatterplot(x = exp2['slope'], y = exp2['predicted_slope'], ax = ax[1,1], label = '$S_c = 1.15$', c = '#56B4E9', s = 70, legend = True)
sns.scatterplot(x = exp3['slope'], y = exp3['predicted_slope'], ax = ax[1,1], label = '$S_c = 1.4$', c = '#009E73', s = 70, legend = True)
ax[1,1].plot(stemp, stemp, color = 'k', linewidth = 1)
ax[1,1].set_xlabel('Cone Slope (m/m)', fontsize = 12)
ax[1,1].set_ylabel('Predicted Slope (m/m)', fontsize = 12)
"""

sns.scatterplot(x = exp1['l'], y = exp1['resid_slope'], ax = ax[1,1], label = '$S_c = 0.9$', c = '#E69F00', s = 70, legend = True)
sns.scatterplot(x = exp2['l'], y = exp2['resid_slope'], ax = ax[1,1], label = '$S_c = 1.15$', c = '#56B4E9', s = 70, legend = True)
sns.scatterplot(x = exp3['l'], y = exp3['resid_slope'], ax = ax[1,1], label = '$S_c = 1.4$', c = '#009E73', s = 70, legend = True)
ax[1,1].set_xlabel('$\\ell$ (m)', fontsize = 11)
ax[1,1].set_ylabel("Predicted Slope Errors, $\sigma_{\\theta}$ (m/m)", fontsize = 11)
ax[1,1].set_xlim([10**(-2)-0.001, 1.1])
ax[1,1].set_xscale('log')


ax[0,0].annotate("(a)", xy = (-0.1, 1.05), xycoords = "axes fraction", fontsize = 12)
ax[0,1].annotate("(b)", xy = (-0.1, 1.05), xycoords = "axes fraction", fontsize = 12)
ax[1,0].annotate("(c)", xy = (-0.1, 1.05), xycoords = "axes fraction", fontsize = 12)
ax[1,1].annotate("(d)", xy = (-0.1, 1.05), xycoords = "axes fraction", fontsize = 12)

ax[0,0].annotate("$z' = 10^{c_1} \ell^{k_1}$", xy = (0.5, 0.8), xycoords = 'axes fraction', fontsize = 14)
ax[0,0].annotate("$k_1 \\approx -0.11 S_c - 0.21$", xy = (0.5, 0.74), xycoords = 'axes fraction', fontsize = 12)
ax[0,0].annotate("$c_1 \\approx 0.10 S_c - 0.69$", xy = (0.5, 0.68), xycoords = 'axes fraction', fontsize = 12)

ax[1,0].annotate("$\\overline{\\theta} = 10^{c_2} \ell^{k_2}$", xy = (0.5, 0.8), xycoords = 'axes fraction', fontsize = 14)
ax[1,0].annotate("$k_2 \\approx -0.16 S_c - 0.25$", xy = (0.5, 0.74), xycoords = 'axes fraction', fontsize = 12)
ax[1,0].annotate("$c_2 \\approx 0.12 S_c - 0.93$", xy = (0.5, 0.68), xycoords = 'axes fraction', fontsize = 12)

plt.tight_layout()
plt.savefig('height-slope-vs-ell.png', dpi = 300, bbox_inches = 'tight')

plt.show()

In [ ]:
s = np.linspace(0.9, 1.4, 10)
m, b, p, r, se = linregress(np.array([0.9, 1.15, 1.4]),np.array([m1, m2, m3]))
print('slope formula m=', m, 'b=', b)

plt.plot(s, m*s + b)
#plt.plot(s, s -0.5)
plt.plot(np.array([0.9, 1.15, 1.4]),np.array([m1, m2, m3]), 'o')

In [ ]:
s = np.linspace(0.9, 1.4, 10)
mk, bk, pk, rk, sek = linregress(np.array([0.9, 1.15, 1.4]),np.array([b1, b2,b3]))
print('intercept formula m=', mk, 'b=', bk)

plt.plot(s, mk*s + bk)
#plt.plot(s, s -0.5)
plt.plot(np.array([0.9, 1.15, 1.4]),np.array([b1, b2, b3]), 'o')

In [ ]:
s = np.linspace(0.9, 1.4, 10)
ms, bs, ps, rs, ses = linregress(np.array([0.9, 1.15, 1.4]),np.array([m1s, m2s,m3s]))
print('slope formula m=', ms, 'b=', bs)
plt.plot(s, ms*s + bs)
#plt.plot(s, s -0.5)
plt.plot(np.array([0.9, 1.15, 1.4]),np.array([m1s, m2s,m3s]), 'o')

In [ ]:
s = np.linspace(0.9, 1.4, 10)
mks, bks, pks, rks, seks = linregress(np.array([0.9, 1.15, 1.4]),np.array([b1s, b2s, b3s]))
print('intercept formula m=', mks, 'b=', bks)
plt.plot(s, mks*s + bks)
#plt.plot(s, s -0.5)
plt.plot(np.array([0.9, 1.15, 1.4]), np.array([b1s, b2s, b3s]), 'o')

In [ ]:
exp1['predicted_height'] = 10**(mk*exp1['S_c'] + bk)*exp1['l']**(m*exp1['S_c'] + b)
exp2['predicted_height'] = 10**(mk*exp2['S_c'] + bk)*exp2['l']**(m*exp2['S_c'] + b)
exp3['predicted_height'] = 10**(mk*exp3['S_c'] + bk)*exp3['l']**(m*exp3['S_c'] + b)

exp1['predicted_slope'] = 10**(mks*exp1['S_c'] + bks)*exp1['l']**(ms*exp1['S_c'] + bs)
exp2['predicted_slope'] = 10**(mks*exp2['S_c'] + bks)*exp2['l']**(ms*exp2['S_c'] + bs)
exp3['predicted_slope'] = 10**(mks*exp3['S_c'] + bks)*exp3['l']**(ms*exp3['S_c'] + bs)

In [ ]:
xtemp = np.linspace(0.3, 1.2)
plt.plot(exp1['cone_height'], exp1['predicted_height'], '.')
plt.plot(exp2['cone_height'], exp2['predicted_height'], '.')
plt.plot(exp3['cone_height'], exp3['predicted_height'], '.')
plt.plot(xtemp, xtemp)

In [ ]:
stemp = np.linspace(0.2, 1.2)
plt.plot(exp1['slope'], exp1['predicted_slope'], '.')
plt.plot(exp2['slope'], exp2['predicted_slope'], '.')
plt.plot(exp3['slope'], exp3['predicted_slope'], '.')
plt.plot(stemp, stemp)

In [ ]:
dflist = [exp1, exp2, exp3]
residh = 0
resids = 0
for df in dflist:
    print(np.mean(np.abs(df['resid_heights'])))
    residh += np.mean(np.abs(df['resid_heights']))
    print(np.mean(np.abs(df['resid_slope'])))
    resids += np.mean(np.abs(df['resid_slope']))

print(residh/3)
print(resids/3)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(x = exp1['l'], y = exp1['resid_heights'], ax = ax, label = '$S_c = 0.9$', c = '#E69F00', s = 70, legend = True)
sns.scatterplot(x = exp2['l'], y = exp2['resid_heights'], ax = ax, label = '$S_c = 1.15$', c = '#56B4E9', s = 70, legend = True)
sns.scatterplot(x = exp3['l'], y = exp3['resid_heights'], ax = ax, label = '$S_c = 1.4$', c = '#009E73', s = 70, legend = True)
ax.set_xlabel('$\\ell$')
ax.set_ylabel("Residual Errors, $\sigma_{z'}$ (m)")
ax.set_xlim([10**(-2)-0.001, 1.1])
plt.xscale('log')
#plt.grid()